# Preprocessing datasets

File dedicated to preprocessing datasets to preprare them for use in the final model. 

In [10]:
# importing packages
import pandas as pd
import os
import problem

## US GDP data

In [24]:
us_gdp = pd.read_csv(os.path.join('raw_add_data', 'gdp_data.csv'), na_values='(D)')
us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN


The first step is to associate the column GeoName with the list of airports from the original problem. 

In [25]:
cities = us_gdp['GeoName'].unique()
print(f'Total cities: {len(cities)}')
print(f'The cities are: \n{cities}')


Total cities: 18
The cities are: 
['Atlanta-Sandy Springs-Alpharetta, GA (Metropolitan Statistical Area)'
 'Austin-Round Rock-Georgetown, TX (Metropolitan Statistical Area)'
 'Boston-Cambridge-Newton, MA-NH (Metropolitan Statistical Area)'
 'Charlotte-Concord-Gastonia, NC-SC (Metropolitan Statistical Area)'
 'Chicago-Naperville-Elgin, IL-IN-WI (Metropolitan Statistical Area)'
 'Denver-Aurora-Lakewood, CO (Metropolitan Statistical Area) *'
 'Detroit-Warren-Dearborn, MI (Metropolitan Statistical Area)'
 'Houston-The Woodlands-Sugar Land, TX (Metropolitan Statistical Area)'
 'Las Vegas-Henderson-Paradise, NV (Metropolitan Statistical Area)'
 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)'
 'Miami-Fort Lauderdale-Pompano Beach, FL (Metropolitan Statistical Area)'
 'Minneapolis-St. Paul-Bloomington, MN-WI (Metropolitan Statistical Area)'
 'New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)'
 'Orlando-Kissimmee-Sanford, FL (Metropolitan Statistical Are

In [26]:
#retrieve name of the first city from GeoName column
us_gdp['city'] = us_gdp['GeoName'].apply(lambda city: city.split('-')[0])
cities = us_gdp['city'].unique()
print('The values in the column city are:\n', cities) 

The values in the column city are:
 ['Atlanta' 'Austin' 'Boston' 'Charlotte' 'Chicago' 'Denver' 'Detroit'
 'Houston' 'Las Vegas' 'Los Angeles' 'Miami' 'Minneapolis' 'New York'
 'Orlando' 'Philadelphia' 'Phoenix' 'San Francisco' 'Seattle']


We can check the correspondance between the cities and the airports from the file `city_ariport.csv`

In [27]:
city_airport = pd.read_csv(os.path.join('raw_add_data', 'city_airport.csv'), header=None)
city_airport.columns = ['airport', 'city']
print(city_airport)

   airport           city
0      ATL        Atlanta
1      BOS         Boston
2      CLT      Charlotte
3      ORD        Chicago
4      DEN         Denver
5      DTW        Detroit
6      DFW         Austin
7      IAH        Houston
8      LAS      Las Vegas
9      LAX    Los Angeles
10     MIA          Miami
11     MSP    Minneapolis
12     JFK       New York
13     LGA       New York
14     EWR       New York
15     MCO        Orlando
16     PHL   Philadelphia
17     PHX        Phoenix
18     SFO  San Francisco
19     SEA        Seattle


In [28]:
merged_us_gdp = pd.merge(us_gdp, city_airport, on='city')
merged_us_gdp.head()

,GeoFips,GeoName,LineCode,Description,2011,2012,2013,city,airport
0,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",1.0,All industry total,277405509.0,287570424.0,299918186.0,Atlanta,ATL
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",2.0,Private industries,250779823.0,260962774.0,273113553.0,Atlanta,ATL
2,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",3.0,"Agriculture, forestry, fishing and hunting",NaN,NaN,602096.0,Atlanta,ATL
3,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",6.0,"Mining, quarrying, and oil and gas extraction",267771.0,228417.0,231368.0,Atlanta,ATL
4,12060,"Atlanta-Sandy Springs-Alpharetta, GA (Metropol...",10.0,Utilities,3704264.0,3582854.0,NaN,Atlanta,ATL


The columns `city`, `GeoFips` and `GeoName` are no longer useful. The column `Description` is encode in `LineCode` and can also be deleted.

In [9]:
del (merged_us_gdp['GeoFips'], merged_us_gdp['GeoName'], merged_us_gdp['Description'],
     merged_us_gdp['city'])


In [10]:
merged_us_gdp.head()

,LineCode,2011,2012,2013,airport
0,1.0,277405509.0,287570424.0,299918186.0,ATL
1,2.0,250779823.0,260962774.0,273113553.0,ATL
2,3.0,NaN,NaN,602096.0,ATL
3,6.0,267771.0,228417.0,231368.0,ATL
4,10.0,3704264.0,3582854.0,NaN,ATL


In [11]:
#melting data frame to obtain years as a factor
melted_us_gdp = merged_us_gdp.melt(id_vars=['LineCode', 'airport'], 
                                   value_vars=['2011', '2012', '2013'],
                                   var_name='year', value_name='gdp')
melted_us_gdp.head()

,LineCode,airport,year,gdp
0,1.0,ATL,2011,277405509.0
1,2.0,ATL,2011,250779823.0
2,3.0,ATL,2011,NaN
3,6.0,ATL,2011,267771.0
4,10.0,ATL,2011,3704264.0


In [12]:
#converting dates to datetime
melted_us_gdp['year'] = pd.to_datetime(melted_us_gdp['year'], format='%Y')
melted_us_gdp.set_index('year', inplace=True)
melted_us_gdp.head()

,LineCode,airport,gdp
year,,,
2011-01-01,1.0,ATL,277405509.0
2011-01-01,2.0,ATL,250779823.0
2011-01-01,3.0,ATL,NaN
2011-01-01,6.0,ATL,267771.0
2011-01-01,10.0,ATL,3704264.0


In [19]:
melted_us_gdp.to_csv(os.path.join('processed_add_data', 'gdp_data_processed.csv'))

## US holiday data

In [17]:
us_hol = pd.read_csv(os.path.join('raw_add_data', 'usholidays.csv'), usecols=['Date', 'Holiday'])
us_hol['Date'] = pd.to_datetime(us_hol['Date']) # convert dates to datetime
us_hol.set_index('Date', inplace=True)
us_hol.head()

,Holiday
Date,
2010-12-31,New Year's Day
2011-01-17,"Birthday of Martin Luther King, Jr."
2011-02-21,Washington's Birthday
2011-05-30,Memorial Day
2011-07-04,Independence Day


To select the relevant dates, we rely on the train and test datasets

In [18]:
x_train, _ = problem.get_train_data()
x_test, _ = problem.get_test_data()

max_date = max(x_test['DateOfDeparture'].max(), x_train['DateOfDeparture'].max())
min_date = min(x_test['DateOfDeparture'].min(), x_train['DateOfDeparture'].min())

print(f'The max date is: {max_date}')
print(f'The min date is: {min_date}')

The max date is: 2013-03-05
The min date is: 2011-09-01


In [20]:
us_hol = us_hol[min_date : max_date]
us_hol.to_csv(os.path.join('processed_add_data', 'usholidays_processed.csv'))

## US demographic data

In [ ]:
#https://stats.oecd.org/Index.aspx?DataSetCode=CITIES
#gives working population (15-65 yo) by metropolitan areas

In [59]:
#loading and renaming dataset
working_pop = pd.read_csv(os.path.join('raw_add_data', 'working_age_pop.csv'), usecols=['Metropolitan areas', 'Year', 'Value'])
working_pop = working_pop.rename(columns={'Metropolitan areas':'city', 'Year':'year', 'Value':'work_pop'})
working_pop.head()

,city,year,work_pop
0,New York (Greater),2011,13415543
1,New York (Greater),2012,13425475
2,New York (Greater),2013,13435986
3,Los Angeles (Greater),2011,11781575
4,Los Angeles (Greater),2012,11854951


In [60]:
#join working_pop to city_airport to get add airport belonging to metro_area
#clean metropolitan area names 
working_pop['city'] = working_pop['city'].str.replace("Greater", "")
working_pop['city'] = working_pop['city'].str.replace("\\(\\)", "")
working_pop['city'] = working_pop['city'].str.rstrip()

In [64]:
working_pop_airport = pd.merge(working_pop, city_airport, how='left',  on='city')
working_pop_airport.head()

,city,year,work_pop,airport
0,New York,2011,13415543,JFK
1,New York,2011,13415543,LGA
2,New York,2011,13415543,EWR
3,New York,2012,13425475,JFK
4,New York,2012,13425475,LGA
